In [ ]:
#These parameters are updated externally... Do not move them
msname = "1491291289.1ghz.1.1ghz.4hrs"
outputdir = "output"
msindex = 0

# Observation report

## Table of Contents: <a name="toc"></a>

1. [Observation Info](#obsInfo)
2. [Flagging Statistics](#flagStats)
3. [Images](#images)
4. [HI cubes](#hiCubes)
5. [Cross calibration diagnostics](#ccal)
    1. [Calibration solutions](#crossCal)
    2. [Visibility diagnostic plots](#visdiags)

In [ ]:
from IPython.display import HTML
HTML("""
        <script>
            code_show=true; 
            function code_toggle() {
                if (code_show){
                    $('div.input').hide();
                } else {
                    $('div.input').show();
                }
                code_show = !code_show
            } 
            $( document ).ready(code_toggle);
        </script>

        The raw code for this notebook is by default hidden for easier reading. To toggle on/off the raw code, click <a href="javascript:code_toggle()">here</a>.
     """)


In [ ]:
HTML("<style>.container { width:100% !important; }</style>")

In [ ]:
from os import path, devnull
import sys
fdevnull = open(devnull, "w")
sys.stdout = fdevnull
sys.stderr = fdevnull
            
import numpy as np
from matplotlib import pyplot as plt
from glob import glob
import aplpy

import ipywidgets
from ipywidgets import  Tab, Accordion, HTML, HBox
def Image(f):
    with open(f, "rb") as fdata: 
        return ipywidgets.Image(value=fdata.read(), format='png')
from IPython.display import display, IFrame
import base64


In [ ]:
def safe_load(filename, mimetype, *args, **kwargs):
    try:
        if mimetype == "image/png":
            return Image(filename)
        elif mimetype == "text/plain":
            with open(filename, 'r') as f:
                txt = f.read()
            packet = base64.encodestring(txt)
            return HTML("<iframe src=\"data:%s;base64,%s\" width=\"%s\" height=\"%s\"/>" % (mimetype, 
                    packet, args[0], args[1]))
        elif mimetype == "application/fits":
            f = aplpy.FITSFigure(filename, 
                                 figsize=kwargs.pop("figsize", (15,15)))
            label = path.basename(filename)
            f.add_label(0.5, 0.98, label, relative=True)
            f.show_colorscale(*args, **kwargs)
            f.add_colorbar()
            f.add_beam()
            f.beam.set_color('magenta')
            f.beam.set_hatch('+')
            f.save("/tmp/caracal_aplpy_img.png")
            f.close()
            return Image("/tmp/caracal_aplpy_img.png")
        elif mimetype == "text/html":
            with open(filename, 'r') as f:
                txt = f.read()
            packet = base64.encodestring(txt)
            return HTML("<iframe src=\"data:%s;base64,%s\" width=\"%s\" height=\"%s\"/>" % (mimetype, 
                    packet, args[0], args[1]))
        else:
            raise ValueError("unsupported type")
    except Exception, e:
        return HTML("<p style='color:#ff0000; font-weight:bold'>Could not load specified files. "
                    "This step may not have run, or there was a trainsmash. Check the logs</p>")

# Observation info <a class="anchor" id="obsInfo"></a>

In [ ]:
HTML("<p style='text-align: right;'> <a href='#toc'>Back to Top</a> </p>")

In [ ]:
display(safe_load("%s/caracal-%s-obsinfo.txt" % (outputdir, msname), "text/plain", "100%", 300))

# Flagging statistics <a class="anchor" id="flagStats"></a>

In [ ]:
HTML("<p style='text-align: right;'> <a href='#toc'>Back to Top</a> </p>")

In [ ]:
flaglog_tab = Tab()
def tryload_flaglog(fname, title):
    children = list(flaglog_tab.children)
    children.append(safe_load(fname, "text/plain", "100%", 300))
    flaglog_tab.children = children
    flaglog_tab.set_title(len(children) - 1, title)
tryload_flaglog("%s/log-flagging_summary_flagging_%d_.txt" % (outputdir, msindex), "Precalibration flagging")
tryload_flaglog("%s/log-flagging_summary_crosscal_%d.txt" % (outputdir, msindex), "After apply calibration")
tryload_flaglog("%s/log-flagging_summary_flagging_%d_corr.txt" % (outputdir, msindex), "Target flagging")
tryload_flaglog("%s/log-flagging_summary_image_selfcal_%d.txt" % (outputdir, msindex), "After self calibration")
tryload_flaglog("%s/log-flagging_summary_image_HI_%d.txt" % (outputdir, msindex), "After HI imaging")
display(flaglog_tab)

# Images <a class="anchor" id="images"></a>

In [ ]:
HTML("<p style='text-align: right;'> <a href='#toc'>Back to Top</a> </p>")

In [ ]:
velz = zip(glob("%s/*-MFS-image.fits" % outputdir), glob("%s/*-MFS-residual.fits" % outputdir))
if len(velz) == 0:
    display(HTML("<p style='color:#ff0000; font-weight:bold'>Could not load any restored images. "
                    "This step may not have run, or there was a trainsmash. Check the logs</p>"))
children = []
titles = []
for im, res in velz:
    acchildren = []
    acchildren.append(safe_load(im,
                                "application/fits", 
                                pmin=0, pmax=99.99, cmap="cubehelix", interpolation="bicubic", figsize=(10,10)))
    acchildren.append(safe_load(res,
                                "application/fits", 
                                pmin=0, pmax=99.99, cmap="cubehelix", interpolation="bicubic", figsize=(10,10)))
    titles.append(path.basename(im).replace("-MFS-image.fits", ""))
    accordion = Accordion(acchildren)
    accordion.set_title(0, 'Image')
    accordion.set_title(1, 'Residual')
    children.append(accordion)

children.append(safe_load("%s/caracal-pipeline.fullrest.fits" % outputdir,
                          "application/fits", 
                          pmin=0.1, pmax=99.99, cmap="cubehelix", interpolation="bicubic", figsize=(10,10)))
titles.append("Full restore")
    
tab = Tab()
tab.children = children
for i, title in zip(xrange(len(children)), titles): tab.set_title(i, title)
display(tab)


# HI Cubes 

In [ ]:
HTML("<p style='text-align: right;'> <a href='#toc'>Back to Top</a> </p>")

!!!TODO!!!

# Cross-calibration diagnostics <a class="anchor" id="ccal"></a>

## Calibration solutions <a class="anchor" id="crossCal"></a>

In [ ]:
HTML("<p style='text-align: right;'> <a href='#toc'>Back to Top</a> </p>")

In [ ]:
tab = Tab()
tab.children = [
    safe_load("%s/caracal-%s-1gc1-K0.html" % (outputdir, msname), "text/html", "100%", 700),
    safe_load("%s/caracal-%s-1gc1-B0.html" % (outputdir, msname), "text/html", "100%", 700),
    safe_load("%s/caracal-%s-1gc1-F0.html" % (outputdir, msname), "text/html", "100%", 700),
]
titles = ["delay","bandpass","fluxscale"]
for i, title in zip(xrange(len(children)), titles): tab.set_title(i, title)
display(tab)

## Visibility diagnostic plots <a class="anchor" id="visdiags"></a>

In [ ]:
HTML("<p style='text-align: right;'> <a href='#toc'>Back to Top</a> </p>")

In [ ]:
visdiags_tab = Tab()
def visdiags_ldimage(fnames, accordion_title, subtitle, maintitle):
    children = list(visdiags_tab.children)
    titles = [visdiags_tab.get_title(i) for i in xrange(len(children))]
    if maintitle in titles:
        mainpg_indx = titles.index(maintitle)
        subpg_children = list(visdiags_tab.children[mainpg_indx].children)
        titles_subpages = [visdiags_tab.children[mainpg_indx].get_title(i) for i in xrange(len(subpg_children))]
        if subtitle in titles_subpages:
            subpg_indx = titles_subpages.index(subtitle)
            accordion = visdiags_tab.children[mainpg_indx].children[subpg_indx]
            if len(fnames) == 0:
                accordion_children = [HTML("<p style='color:#ff0000; font-weight:bold'> "
                    "This step may not have run, or there was a trainsmash. Check the logs</p>")]
            else:
                accordion_children = [safe_load(f, "image/png") for f in fnames]
            accordion.children = list(accordion.children) + [HBox(accordion_children)]
            accordion.set_title(len(accordion.children) - 1, accordion_title)
        else:
            subpages = visdiags_tab.children[mainpg_indx]
            if len(fnames) == 0:
                accordion_children = [HTML("<p style='color:#ff0000; font-weight:bold'> "
                    "This step may not have run, or there was a trainsmash. Check the logs</p>")]
            else:
                accordion_children = [safe_load(f, "image/png") for f in fnames]
            accordion = Accordion([HBox(accordion_children)])
            accordion.set_title(len(accordion.children) - 1, accordion_title)
            accordion.set_title(-1, "value")
            subpages.children = list(subpages.children) + [accordion]
            subpages.set_title(len(subpages.children) - 1, subtitle)
    else:
        subpages = Tab()
        if len(fnames) == 0:
            accordion_children = [HTML("<p style='color:#ff0000; font-weight:bold'> "
                "This step may not have run, or there was a trainsmash. Check the logs</p>")]
        else:
            accordion_children = [safe_load(f, "image/png") for f in fnames]
        accordion = Accordion([HBox(accordion_children)])
        accordion.set_title(len(accordion.children) - 1, accordion_title)
        subpages.children = [accordion]
        visdiags_tab.children = list(visdiags_tab.children) + [subpages]
        visdiags_tab.set_title(len(visdiags_tab.children) - 1, maintitle)
        subpages.set_title(len(subpages.children) - 1, subtitle)
visdiags_ldimage(glob("%s/caracal-%s-1gc1-bpcal-scan-ap_*.png" % 
                      (outputdir, msname)),
                 "coloured by scan", "Bandpass Field", "Amp v Phase")

visdiags_ldimage(glob("%s/caracal-%s-1gc1-bpcal-baseline-ap_*.png" % 
                      (outputdir, msname)),
                 "coloured by baseline", "Bandpass Field", "Amp v Phase")
visdiags_ldimage(glob("%s/caracal-%s-1gc1-gcal-scan-ap_*.png" % 
                      (outputdir, msname)),
                 "coloured by scan", "Gain Field", "Amp v Phase")
visdiags_ldimage(glob("%s/caracal-%s-1gc1-gcal-baseline-ap_*.png" % 
                      (outputdir, msname)),
                 "coloured by baseline", "Gain Field", "Amp v Phase")
visdiags_ldimage(glob("%s/caracal-%s-1gc1-bpcal-scan-reim_*.png" % 
                      (outputdir, msname)),
                 "coloured by scan", "Bandpass Field", "Real v Imaginary")
visdiags_ldimage(glob("%s/caracal-%s-1gc1-bpcal-baseline-reim_*.png" % 
                      (outputdir, msname)),
                 "coloured by baseline", "Bandpass Field", "Real v Imaginary")
visdiags_ldimage(glob("%s/caracal-%s-1gc1-gcal-scan-reim_*.png" % 
                      (outputdir, msname)),
                 "coloured by scan", "Gain Field", "Real v Imaginary")
visdiags_ldimage(glob("%s/caracal-%s-1gc1-gcal-baseline-reim_*.png" % 
                      (outputdir, msname)),
                 "coloured by baseline", "Gain Field", "Real v Imaginary")
visdiags_ldimage(glob("%s/caracal-%s-1gc1-bpcal-ampuvwave_*.png" % 
                      (outputdir, msname)),
                 "amplitude", "Bandpass Field", "UVwave")
visdiags_ldimage(glob("%s/caracal-%s-1gc1-gcal-ampuvwave_*.png" % 
                      (outputdir, msname)),
                 "amplitude", "Gain Field", "UVwave")
visdiags_ldimage(glob("%s/caracal-%s-1gc1-bpcal-phaseuvwave_*.png" % 
                      (outputdir, msname)),
                 "phase", "Bandpass Field", "UVwave")
visdiags_ldimage(glob("%s/caracal-%s-1gc1-gcal-phaseuvwave_*.png" % 
                      (outputdir, msname)),
                 "phase", "Gain Field", "UVwave")
visdiags_ldimage(glob("%s/caracal-%s-1gc1-bpcal-ampscan_*.png" % 
                      (outputdir, msname)),
                 "coloured by baseline", "Bandpass Field", "Amp v Scan")
visdiags_ldimage(glob("%s/caracal-%s-1gc1-gcal-ampscan_*.png" % 
                      (outputdir, msname)),
                 "coloured by baseline", "Gain Field", "Amp v Scan")
visdiags_ldimage(glob("%s/caracal-%s-1gc1-target-ampscan_*.png" % 
                      (outputdir, msname)),
                 "coloured by baseline", "Target Field", "Amp v Scan")
velz = glob("%s/%s.ms-AUTOCORR-FIELD-*-CORR-*.png" % 
                      (outputdir, msname))
for v in velz:
    title = "Correlation {0:s}".format(v.split("-CORR-")[1])
    field = v.split("FIELD-")[1].split("-CORR")[0]
    visdiags_ldimage([v],
                     title, field, "Auto Power")
velz = glob("%s/%s.ms-PHASE-FIELD-*-CORR-*" % 
                      (outputdir, msname))
for v in velz:
    title = "Correlation {0:s}".format(v.split("-CORR-")[1])
    field = v.split("FIELD-")[1].split("-CORR")[0]
    visdiags_ldimage([v],
                     title, field, "Phase Spectra")
display(visdiags_tab)